# Custom LINCS random forest

In [1]:
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy

In [23]:
X, y = datasets.make_classification(n_samples=20000, 
                                            n_features=36, 
                                            n_informative=10, 
                                            n_redundant=6, 
                                            n_repeated=0, 
                                            n_classes=2, 
                                            n_clusters_per_class=2, 
                                            weights=None, 
                                            flip_y=0.01, 
                                            class_sep=1.0, 
                                            hypercube=True, 
                                            shift=0.0, 
                                            scale=1.0, 
                                            shuffle=True, 
                                            random_state=1)

In [336]:
X.shape

(20000, 36)

We want to randomly remove data from X in order to simulate the missing data from our LINCS classification problem. Basically every sample has **4 metrics x 9 cell lines = 36 total features**. However, not all samples are tested in all cell lines, but we will say that have at minimum data from four cell lines. This should be made a variable. 

In [4]:
# first assign which cell lines each sample was tested in
min_n_cells = 4
max_n_cells = 9
n_cells_ = np.random.randint(min_n_cells, max_n_cells, len(y))
n_missing_cells_ = 9 - n_cells_

In [5]:
# remove features from each sample's missing cell lines
X_df = pd.DataFrame(X)

for index in range(len(X)):
    n_missing_cells = n_missing_cells_[index]
    
    # randomly choose which cells lines are mising
    missing_cell_lines = np.random.choice(np.arange(9),n_missing_cells, replace=False)
    
    # convert this to the missing feature indeces
    missing_feature_idx = np.array([ 4*i + np.array([0, 1, 2, 3]) for i in missing_cell_lines ]).reshape(1,-1)[0]
    
    # remove the feature values
    X_df.set_value(index, missing_feature_idx, np.NaN)

In [6]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.209149,1.192170,-1.164384,0.484403,-0.456668,1.353219,-1.084165,-0.510960,-1.074072,-0.376052,...,-2.528943,0.326389,NaN,NaN,NaN,NaN,-0.308625,-0.476750,-1.852631,-0.192746
1,-0.724587,0.857552,-0.282390,-0.872162,-1.295439,-1.104565,1.008983,1.783967,0.331429,0.038724,...,NaN,NaN,NaN,NaN,NaN,NaN,1.679008,-0.605346,2.459989,-1.541449
2,2.064542,1.273224,3.966204,0.601887,0.774822,-0.204536,0.246851,-1.001860,NaN,NaN,...,NaN,NaN,-1.064790,-0.591569,0.717058,-1.076956,2.132901,0.559781,-1.215091,0.669849
3,NaN,NaN,NaN,NaN,0.123821,1.134245,-4.068682,-0.384080,1.600720,1.042394,...,-7.302279,0.459751,-0.754988,0.204575,-2.347061,1.891265,-4.264726,-1.290770,0.073044,-0.258931
4,NaN,NaN,NaN,NaN,-1.244846,0.978361,-0.988202,1.456638,-0.067727,0.233124,...,NaN,NaN,-0.776104,0.527975,-1.371177,0.927110,-1.722779,0.150167,-0.023199,-2.650051


Ok, so now we have a dataset with missing values that mimic the missing data we have in our LINCS dataset. Now we have to construct our custom Random Forest implementation that elegantly handles the missing data.

In [13]:
X_missing = X_df.values
X_not_missing = ~np.isnan(X_missing)
num_cells_not_missing = np.count_nonzero(X_not_missing, axis=1) / 4
print(num_cells_not_missing)
np.min(num_cells_not_missing)

[6. 4. 6. ... 6. 8. 5.]


4.0

### Let's try a classic SKLEARN random forest classifier

In [40]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion='gini',
                                n_estimators=10,
                                random_state=1,
                                n_jobs=-1)
forest.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=1,
            verbose=0, warm_start=False)

In [46]:
#[ tree.score(X,y) for tree in forest.estimators_ ]

In [283]:
[ tree.predict(X) for tree in forest.estimators_ ]

[array([1., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 1., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 0.]),
 array([0., 0., 0., ..., 1., 0., 1.])]

In [355]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

LRF = LincsRandomForestClassifier(n_cells_per_forest=4)
LRF.fit(X_missing, y)

In [354]:
import scipy
import itertools

class LincsRandomForestClassifier(object):
    
    "WE ASSUME THE DATA IS GROUPED BY CELL LINE AND HAS 4 FEATURES PER CELL LINE"
   
    def __init__(self, n_cells_per_forest, n_estimators_per_forest=10, max_depth=None, max_features="auto", random_state=1):
        self.n_cells_per_forest = n_cells_per_forest
        self.n_estimators_per_forest = n_estimators_per_forest
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        
    def fit(self, X, y):
        '''
        Train several random forests, each one on a different
        subset of cells. Store forests in a dictionary called
        self.forests.
        '''
        # make sure we have enough data to work with
        min_num_cells = self.get_min_num_cells(X)
        assert min_num_cells >= self.n_cells_per_forest, "Too much missing data for n_cells_per_forest = %s. (Some samples only tested in %d cells)" % \
                                                         (self.n_cells_per_forest, min_num_cells)
        
        # generate cell subsets for training
        total_num_cells = int(X.shape[1] / 4) # THIS IS HARDCODED IN
        cell_subsets = itertools.combinations(np.arange(total_num_cells), self.n_cells_per_forest)
        
        # initialize dictionary to hold the forests
        self.forests = {}
        
        # train forest on each subset
        for cell_subset in cell_subsets:
            
            # find samples that have complete data from the cell subset
            cell_subset_idx = np.array([ 4*i + np.array([0, 1, 2, 3])for i in cell_subset ]).reshape(1,-1)[0].astype(int)
            cell_subset_data = X[:,cell_subset_idx]
            bad_sample_idx = np.isnan(cell_subset_data).any(axis=1)
            good_samples = cell_subset_data[~bad_sample_idx]
            good_labels = y[~bad_sample_idx]
            
            # train and store a RF classifier on this training subset
            # print('Growing forest for cell subset: %s' % str(cell_subset))
            forest = RandomForestClassifier(criterion='gini',
                                            n_estimators=self.n_estimators_per_forest,
                                            max_depth=self.max_depth,
                                            max_features=self.max_features,
                                            random_state=self.random_state,
                                            n_jobs=-1)
            forest.fit(good_samples, good_labels)
            self.forests[cell_subset] = forest            

        
    def get_min_num_cells(self, X):
        '''
        Calculate the minimum number of cells any sample has data for
        ASSUMES EACH CELL LINE HAS 4 FEATURES
        '''
        X_not_missing = ~np.isnan(X)
        num_cells_not_missing = np.count_nonzero(X_not_missing, axis=1) / 4
        min_num_cells = np.min(num_cells_not_missing)
        return min_num_cells
    
    def predict_proba(self, X):
        '''
        Return the class probabilities label OF ONE SINGLE SAMPLE FOR FUCKS SAKE
        '''
        # figure out which cell lines we have data for
        non_nan_idx = np.where(np.isnan(X) == False)[0]
        good_cells = (non_nan_idx[np.where(non_nan_idx/4%1 == 0)[0]] / 4).astype(int)
        
        # select appropriate forests and predict
        cell_subsets = itertools.combinations(good_cells, self.n_cells_per_forest)
        tree_predictions_ = []
        for cell_subset in cell_subsets:
            # extract appropriate data
            cell_subset_idx = np.array([ 4*i + np.array([0, 1, 2, 3])for i in cell_subset ]).reshape(1,-1)[0].astype(int)
            cell_subset_data = X[cell_subset_idx].reshape(1,-1) 
            # extract appropriate forest and make prediction
            forest = self.forests[cell_subset]
            tree_predictions = [ tree.predict(cell_subset_data) for tree in forest.estimators_ ]
            tree_predictions_.append(tree_predictions)
        
        # majority vote of all the trees in all the forests
        results = np.array(tree_predictions_).flatten()
        proba = results.sum() / len(results)
        return np.array([1.-proba, proba])
    
    def predict(self, X):
        '''
        Return the predicted class label OF ONE SINGLE SAMPLE FOR FUCKS SAKE
        '''
        class_probabilities = self.predict_proba(X)
        return np.argmax(class_probabilities)
    
    def predict_proba_(self, X):
        '''
        for a multidimentional X
        '''
        proba_ = np.array([ self.predict_proba(x) for x in X ])
        return proba_
    
    def predict_(self, X):
        '''
        for a multidimentional X
        '''
        predicted_classes = np.array([ self.predict(x) for x in X ])
        return predicted_classes

In [188]:
RandomForestClassifier?

In [338]:
a = np.array([1,2,3,2])

In [342]:
np.argmax(a)

2